## Introduction

In this notebook, we will demonstrate how to use the Solcast SDK to retrieve and visualize solar radiation data. We will start by loading the necessary libraries, then proceed to fetch historical data for a specific location, and finally plot the results using XPlot.Plotly.


In [1]:
#r "nuget: Newtonsoft.Json, 13.0.1"
#r "nuget: System.Net.Http"
#r "nuget: XPlot.Plotly, 4.1.0"
#r "nuget: xplot.plotly.interactive"
#r "../../src/Solcast/bin/Debug/net8.0/Solcast.dll"

Installed Packages Newtonsoft.Json, 13.0.1 System.Net.Http, 4.3.4 XPlot.Plotly, 4.1.0 xplot.plotly.interactive, 4.1.0

Loading extensions from `/root/.nuget/packages/xplot.plotly.interactive/4.1.0/lib/net7.0/XPlot.Plotly.Interactive.dll`

Configuring PowerShell Kernel for XPlot.Plotly integration.

Installed support for XPlot.Plotly.

To begin, we load the necessary libraries for using the Solcast SDK:

In [2]:
using System;
using System.Threading.Tasks;
using Solcast;
using Solcast.Clients;

To test the API you can use [unmetered locations](https://docs.solcast.com.au/#unmetered-locations), in the sdk:

In [2]:
var sydney = UnmeteredLocations.Locations["Sydney Opera House"];

Retrieve the historical data and plot the results.

In [3]:
using Newtonsoft.Json;
using System.Linq;
using XPlot.Plotly;


var historicClient = new HistoricClient();

double latitude = sydney.Latitude;
double longitude = sydney.Longitude;
string start = "2022-06-01T14:45:00.000Z";
string duration = "P5D";
string period = "PT5M";

var response = await historicClient.GetRadiationAndWeather(
    latitude: latitude,
    longitude: longitude,
    start: start,
    duration: duration,
    period: period,
    format: "json"
);
Console.WriteLine(response);

var periodEnd = ((IEnumerable<dynamic>)response.Data.EstimatedActuals)
    .Select(d => (DateTime)d["period_end"])
    .ToArray();

var ghi = ((IEnumerable<dynamic>)response.Data.EstimatedActuals)
    .Select(d => (double)d["ghi"])
    .ToArray();

var clearskyGHI = ((IEnumerable<dynamic>)response.Data.EstimatedActuals)
    .Select(d => (double)d["dni"])  // Using dni as clearsky_ghi
    .ToArray();

// Create traces for GHI and Clearsky GHI
var ghiTrace = new Scatter
{
    x = periodEnd,
    y = ghi,
    name = "GHI",
    mode = "lines",
};

var clearskyGhiTrace = new Scatter
{
    x = periodEnd,
    y = clearskyGHI,
    name = "Clearsky GHI",
    mode = "lines",
};

// Plot the chart
var chart = Chart.Plot(new[] { ghiTrace, clearskyGhiTrace });
chart.WithTitle("GHI vs Clearsky GHI");
chart.WithXTitle("Period End");
chart.WithYTitle("Radiation (W/m²)");

display(chart);

Solcast.Utilities.ApiResponse`1[HistoricRadiationAndWeatherResponse]


<!DOCTYPE html>

In [5]:
using Solcast.Clients;

var location = UnmeteredLocations.Locations["Sydney Opera House"];

var forecastClient = new ForecastClient();

var response = await forecastClient.GetRooftopPvPower(
    latitude: location.Latitude,
    longitude: location.Longitude,
    outputParameters: ["pv_power_rooftop"],
    period: "PT5M",
    capacity: 5,
    tilt: 22,
    format: "csv"
);

In [6]:
response.RawResponse

<!doctype html>
<html lang="en" style="">
<head>
<title>GetForecastRooftopPvPower Snapshot of 09/24/2024 05:10:54</title>
<meta charset="utf-8" />
<meta name="viewport" content="width=device-width, initial-scale=1.0" />
<base href="https://api.solcast.com.au/">
<link href="css/ui-highlight.css" rel="stylesheet">
<link href="css/ui.css" rel="stylesheet">
<script type="importmap">
{
    "imports": {
        "vue": "./js/vue.mjs",  
        "@servicestack/vue": "./js/servicestack-vue.mjs",
        "@servicestack/client": "./js/servicestack-client.mjs",
        "core": "./js/core.mjs"
    }
}
</script>
<style>
#header a { text-decoration: underline; font-weight: 600 }
#header i { font-style: normal; font-weight: 600 }
</style>

</head>
<body>
<script>
window.BaseUrl = "https://api.solcast.com.au/"
window.ServiceUrl = "https://api.solcast.com.au/data/forecast/rooftop_pv_power?latitude=-33.856784&amp;longitude=151.215297&amp;capacity=5&amp;period=PT5M&amp;tilt=22&amp;outputParameters=pv_powe